In [70]:
from fredapi import Fred
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import CubicSpline


In [71]:
api_key = "08e5257628ddf030304b107dd5185a26"
fred = Fred(api_key=api_key)

# Set the start and end dates for the data
start_date = '1994-01-01'
end_date = '2014-01-01'

economic_indicators = [
    # Growth Statistics
    "GDPC1", "GDP", "GDPPOT", "GNPDEF", "INDPRO",
    
    # Price and Inflation Indices
    "CPIAUCSL", "CPIAUCSL", "CPILFESL", "PPIACO", "PCUOMFGOMFG", "PCEPI",
    
    # Money Supply Statistics
    "M1SL", "M2SL", "M3SL", "MZMSL", "BASE",
    
    # Interest Rates
    "FEDFUNDS", "DGS10", "TB3MS", "BAA", "DFF",
    
    # Employment Statistics
    "PAYEMS", "UNRATE", "AWHMAN", "CE16OV", "USPRIV",
    
    # Income and Expenditure Statistics
    "DPI", "PCECC96", "TOTALSA", "PSAVERT", "NEWORDER",
    
    # Debt Levels
    "GFDEBTN", "TCMDO",
    
    # Miscellaneous Indicators
    "HOUST", "GPDI", "DCOILWTICO"
]


# Fetch data for multiple indicators
data = pd.DataFrame()
for indicator in economic_indicators:
    try:
        data[indicator] = fred.get_series(indicator, start_date=start_date, end_date=end_date)
    except ValueError as e:
        print(f"Error fetching data for {indicator}: {e}")


In [72]:
fred_indicators = data.loc[start_date:end_date]

In [73]:
daily_date_range = pd.date_range(start=start_date, end=end_date, freq='D')

# Create a new dataframe with daily timestamped rows
daily_data_df = pd.DataFrame({'Date': daily_date_range})

# Set 'Date' as the index for the dataframe
daily_data_df.set_index('Date', inplace=True)

# Merge the original quarterly data with the new daily data
merged_data_df = pd.concat([fred_indicators, daily_data_df], axis=0)

# Sort the merged dataframe by the index (Date) to ensure the dates are in order
merged_data_df.sort_index(inplace=True)



In [74]:
duplicates = merged_data_df.index.duplicated(keep='first')

merged_data_df = merged_data_df[~duplicates]

In [75]:
def cubic_spline_interpolation(quarterly_gdp_data, indicator):
    # Forward-fill missing values in GDP data
    gdp_data = quarterly_gdp_data[indicator].copy()

    # Find the indices of NaN values in the GDP data
    nan_indices = gdp_data.isnull()

    # Find the indices of non-NaN values in the GDP data
    non_nan_indices = ~nan_indices

    # Check if there are at least two non-NaN values to perform interpolation
    if non_nan_indices.sum() < 2:
        # If there are not enough non-NaN values, drop the indicator from the DataFrame
        quarterly_gdp_data.drop(columns=[indicator], inplace=True)
    else:
        # Create a Cubic Spline interpolation object using dates as the known data
        cs = CubicSpline(gdp_data.index[non_nan_indices], gdp_data[non_nan_indices])

        # Use the interpolation object to estimate the missing values
        gdp_data[nan_indices] = cs(gdp_data.index[nan_indices])

    return gdp_data



In [78]:
interpolated_data_df = merged_data_df.copy()

for i, ind in enumerate(merged_data_df.columns):
    interpolated_data_df[ind] = cubic_spline_interpolation(merged_data_df, ind)

percentage_change_df = interpolated_data_df.pct_change()

In [ ]:
percentage_change_df.to_csv('fred_indicators.csv', index=True)